# Import Section
---

In [1]:
"""
This script is used for training a keyword spotting model using TensorFlow
with ipywidgets UI.
"""

import os
import argparse
from pathlib import Path
import shutil
from collections import OrderedDict

import tensorflow as tf
import numpy as np
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

import ipywidgets as widgets
from ipywidgets import Layout, Box, Dropdown, Label, IntSlider
from IPython.display import display, HTML, clear_output

from kws_python import data
from kws_python import models

# Training Section
---

In [2]:
def train(flags, save_cmd_filename):
    """
    Trains a TensorFlow model for keyword spotting.

    Args:
        flags: An object containing various training parameters such as:
            - wanted_words: Comma-separated list of words to recognize.
            - sample_rate: Audio sample rate.
            - clip_duration_ms: Duration of each audio clip in milliseconds.
            - window_size_ms: Duration of each spectrogram timeslice in milliseconds.
            - window_stride_ms: How far to move in time between spectrogram timeslices.
            - dct_coefficient_count: Number of bins to use for the MFCC fingerprint.
            - model_architecture: Type of model architecture to use.
            - model_size_info: List of integers defining the model layers.
            - data_exist: Boolean indicating if data already exists.
            - data_url: URL to download data from.
            - data_dir: Directory to store data.
            - silence_percentage: Percentage of training data that should be silence.
            - unknown_percentage: Percentage of training data that should be unknown words.
            - validation_percentage: Percentage of data to use for validation.
            - testing_percentage: Percentage of data to use for testing.
            - how_many_training_steps: Comma-separated list of number of training steps.
            - learning_rate: Comma-separated list of learning rates.
            - batch_size: Number of samples per batch.
            - eval_step_interval: Number of steps between evaluations.
            - train_dir: Directory to save training checkpoints and logs.
            - summaries_dir: Directory to save TensorBoard summaries.
            - save_cmd_filename: The filename to save the training command.

    This function prepares the model settings, creates the model, processes the audio data, 
    sets up the learning rate schedule and optimizer, and trains the model. It also evaluates 
    the model on test data and saves the results, including the model's MACs and total parameters.
    """

    model_settings = models.prepare_model_settings(
        len(data.prepare_words_list(flags.wanted_words.split(","))), flags.sample_rate, flags.clip_duration_ms, flags.window_size_ms, flags.window_stride_ms, flags.dct_coefficient_count
    )

    # Create the model.
    model = models.create_model(model_settings, flags.model_architecture, flags.model_size_info, True)

    audio_processor = data.AudioProcessor(
        data_exist=flags.data_exist,
        data_url=flags.data_url,
        data_dir=flags.data_dir,
        silence_percentage=flags.silence_percentage,
        unknown_percentage=flags.unknown_percentage,
        wanted_words=flags.wanted_words.split(","),
        validation_percentage=flags.validation_percentage,
        testing_percentage=flags.testing_percentage,
        model_settings=model_settings,
    )

    # We decay learning rate in a constant piecewise way to help learning.
    training_steps_list = list(map(int, flags.how_many_training_steps.split(",")))
    learning_rates_list = list(map(float, flags.learning_rate.split(",")))
    lr_boundary_list = training_steps_list[:-1]  # Only need the values at which to change lr.
    lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=lr_boundary_list, values=learning_rates_list)

    # Specify the optimizer configurations.
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

    train_data = audio_processor.get_data(audio_processor.Modes.TRAINING, flags.background_frequency, flags.background_volume, int((flags.time_shift_ms * flags.sample_rate) / 1000))
    train_data = train_data.repeat().batch(flags.batch_size).prefetch(tf.data.AUTOTUNE)
    val_data = audio_processor.get_data(audio_processor.Modes.VALIDATION)
    val_data = val_data.batch(flags.batch_size).prefetch(tf.data.AUTOTUNE)

    # We train for a max number of iterations so need to calculate how many 'epochs' this will be.
    training_steps_max = np.sum(training_steps_list)
    training_epoch_max = int(np.ceil(training_steps_max / flags.eval_step_interval))

    # Callbacks.
    train_dir = Path(flags.train_dir) / "best"
    train_dir.mkdir(parents=True, exist_ok=True)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=(train_dir / (flags.model_architecture + "_{val_accuracy:.3f}_ckpt")), save_weights_only=True, monitor="val_accuracy", mode="max", save_best_only=True
    )
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=flags.summaries_dir)

    # Save the train model seeting
    src = Path(os.getcwd()) / save_cmd_filename
    dst = Path(os.getcwd()) / Path(flags.train_dir) / save_cmd_filename
    shutil.copy(src, dst)

    # Train the model.
    model.fit(x=train_data, steps_per_epoch=flags.eval_step_interval, epochs=training_epoch_max, validation_data=val_data, callbacks=[model_checkpoint_callback, tensorboard_callback])

    # Test and save the model.
    test_data = audio_processor.get_data(audio_processor.Modes.TESTING)
    test_data = test_data.batch(flags.batch_size)

    test_loss, test_acc = model.evaluate(x=test_data)
    print(f"Final test accuracy : {test_acc*100:.2f}% and test_loss : {test_loss:.4f}")

    # save result record
    forward_pass = tf.function(model.call, input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])
    graph_info = profile(forward_pass.get_concrete_function().graph, options=ProfileOptionBuilder.float_operation())
    # The //2 is necessary since `profile` counts multiply and accumulate
    # as two flops, here we report the total number of multiply accumulate ops
    flops = graph_info.total_float_ops // 2
    total_para = model.count_params()
    print(f"TensorFlow: {tf.__version__}")
    print(f"The MACs of this model: {flops:,}")
    print(f"The total parameters of this model: {total_para:,}")

    test_txt_path = os.path.join(os.getcwd(), flags.train_dir, "result_record.txt")
    with open(test_txt_path, "w", encoding="utf-8") as f:
        f.write(f"Test accuracy: {test_acc}" + "\n")
        f.write(f"MACs: {flops}" + "\n")
        f.write(f"Total Parameters: {total_para}" + "\n")

# Argument Setting
---

In [3]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_exist", type=bool, default=True, help="True will skip download and tar.")
    parser.add_argument("--data_url", type=str, default="http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz", help="Location of speech training data archive on the web.")
    parser.add_argument(
        "--data_dir",
        type=str,
        default="tmp/speech_dataset",
        help="""\
        Where to download the speech training data to.
        """,
    )
    parser.add_argument(
        "--background_volume",
        type=float,
        default=0.1,
        help="""\
        How loud the background noise should be, between 0 and 1.
        """,
    )
    parser.add_argument(
        "--background_frequency",
        type=float,
        default=0.8,
        help="""\
        How many of the training samples have background noise mixed in.
        """,
    )
    parser.add_argument(
        "--silence_percentage",
        type=float,
        default=10.0,
        help="""\
        How much of the training data should be silence.
        """,
    )
    parser.add_argument(
        "--unknown_percentage",
        type=float,
        default=10.0,
        help="""\
        How much of the training data should be unknown words.
        """,
    )
    parser.add_argument(
        "--time_shift_ms",
        type=float,
        default=100.0,
        help="""\
        Range to randomly shift the training audio by in time.
        """,
    )
    parser.add_argument("--testing_percentage", type=int, default=10, help="What percentage of wavs to use as a test set.")
    parser.add_argument("--validation_percentage", type=int, default=10, help="What percentage of wavs to use as a validation set.")
    parser.add_argument(
        "--sample_rate",
        type=int,
        default=16000,
        help="Expected sample rate of the wavs",
    )
    parser.add_argument(
        "--clip_duration_ms",
        type=int,
        default=1000,
        help="Expected duration in milliseconds of the wavs",
    )
    parser.add_argument(
        "--window_size_ms",
        type=float,
        default=30.0,
        help="How long each spectrogram timeslice is",
    )
    parser.add_argument(
        "--window_stride_ms",
        type=float,
        default=10.0,
        help="How long each spectrogram timeslice is",
    )
    parser.add_argument(
        "--dct_coefficient_count",
        type=int,
        default=40,
        help="How many bins to use for the MFCC fingerprint",
    )
    parser.add_argument(
        "--how_many_training_steps",
        type=str,
        # default='15,3',
        default="15000,3000",
        help="How many training loops to run",
    )
    parser.add_argument("--eval_step_interval", type=int, default=400, help="How often to evaluate the training results.")
    parser.add_argument("--learning_rate", type=str, default="0.001,0.0001", help="How large a learning rate to use when training.")
    parser.add_argument(
        "--batch_size",
        type=int,
        default=100,
        help="How many items to train with at once",
    )
    parser.add_argument("--summaries_dir", type=str, default="/tmp/retrain_logs", help="Where to save summary logs for TensorBoard.")
    parser.add_argument(
        "--wanted_words",
        type=str,
        default="yes,no",
        help="Words to use (others will be added to an unknown label)",
    )
    parser.add_argument("--train_dir", type=str, default="/tmp/speech_commands_train", help="Directory to write event logs and checkpoint.")
    parser.add_argument("--model_architecture", type=str, default="dnn", help="What model architecture to use")
    parser.add_argument("--model_size_info", type=int, nargs="+", default=[128, 128, 128], help="Model dimensions - different for various models")

# Widgets Control Section
---

In [4]:
class InitTrainWidgets:
    """
    A class to initialize and manage training and data parameter widgets for a machine learning model.
    Methods
    -------
    folder_num_check(train_loc, dataset_list_check):
        Checks if the number of files in each dataset folder is less than 15.
    create_command(cm_list):
        Creates a command dictionary from the provided list of parameters and saves it to a file.
    show_main():
        Displays the main interface with training and data parameter widgets and interactive controls.
    run_test():
        Reads the saved command file, parses the arguments, and runs the training process.
    """
    def __init__(self):

        form_item_layout = Layout(
            display="flex",
            flex_flow="row",
            justify_content="space-between",
        )

        # train model parameters widgets
        self.a_ta = Dropdown(options=["ds_cnn", "dnn", "cnn", "basic_lstm"])
        self.b_ta = widgets.BoundedIntText(value=10, min=0, max=50.0, step=1, disabled=False)
        self.c_ta = widgets.BoundedIntText(value=10, min=0, max=50.0, step=1, disabled=False)
        self.d_ta = widgets.Text(value="5000,10000,10000", placeholder="Type something", description="String:", disabled=False)
        self.e_ta = widgets.Text(value="0.0005,0.0001,0.00002", placeholder="Type something", description="String:", disabled=False)
        self.f_ta = widgets.IntSlider(value=400, min=100, max=1500, step=100)
        self.g_ta = widgets.IntSlider(value=100, min=50, max=1000, step=50)
        self.h_ta = widgets.Text(value="5 64 10 4 2 2 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1", placeholder="Type something", description="Int:", disabled=False)
        self.i_ta = widgets.Textarea(value="yes,no,up,down,left,right,on,off,stop,go", placeholder="Type something", description="String:", disabled=False)
        self.j_ta = widgets.Text(value="work/DS_CNN/1/retrain_logs", placeholder="Type something", description="String:", disabled=False)
        self.k_ta = widgets.Text(value="work/DS_CNN/1/training", placeholder="Type something", description="String:", disabled=False)

        form_train_items = [
            Box([Label(value="Model Architecture"), self.a_ta], layout=form_item_layout),
            Box([Label(value="Testing percentage"), self.b_ta], layout=form_item_layout),
            Box([Label(value="Validation percentage"), self.c_ta], layout=form_item_layout),
            Box([Label(value="Training Steps"), self.d_ta], layout=form_item_layout),
            Box([Label(value="Learning rates"), self.e_ta], layout=form_item_layout),
            Box([Label(value="Eval step interval"), self.f_ta], layout=form_item_layout),
            Box([Label(value="Batch size"), self.g_ta], layout=form_item_layout),
            Box([Label(value="Model size (dimension)"), self.h_ta], layout=form_item_layout),
            Box([Label(value="Wanted words"), self.i_ta], layout=form_item_layout),
            Box([Label(value="Summaries directory"), self.j_ta], layout=form_item_layout),
            Box([Label(value="Train directory"), self.k_ta], layout=form_item_layout),
        ]

        self.form_box_train_para = Box(
            form_train_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightblue",
                align_items="stretch",
                width="50%",
            ),
        )

        # data parameters widgets
        self.a_da = IntSlider(value=10, min=10, max=50)
        self.b_da = widgets.Checkbox(value=True, disabled=False, indent=False)
        self.c_da = widgets.FloatSlider(value=0.5, min=0.0, max=1.0)
        self.d_da = widgets.FloatSlider(value=0.9, min=0.0, max=1.0)
        self.e_da = widgets.FloatSlider(value=20.0, min=0.0, max=50.0)
        self.f_da = widgets.FloatSlider(value=20.0, min=0.0, max=50.0)
        self.g_da = widgets.FloatSlider(value=200.0, min=50.0, max=500.0, step=10.0)
        self.h_da = widgets.IntSlider(value=16000, min=16000, max=32000, step=16000)
        self.i_da = widgets.IntSlider(value=1000, min=800, max=3000, step=200)
        self.j_da = widgets.IntSlider(value=40, min=10, max=100, step=10)
        self.k_da = widgets.IntSlider(value=20, min=10, max=100, step=10)

        form_data_items = [
            Box([Label(value="DCT coefficient count"), self.a_da], layout=form_item_layout),
            Box([Label(value="Data exist"), self.b_da], layout=form_item_layout),
            Box([Label(value="Background volume"), self.c_da], layout=form_item_layout),
            Box([Label(value="Background frequency"), self.d_da], layout=form_item_layout),
            Box([Label(value="Silence percentage"), self.e_da], layout=form_item_layout),
            Box([Label(value="Unknown percentage"), self.f_da], layout=form_item_layout),
            Box([Label(value="Time shift (ms)"), self.g_da], layout=form_item_layout),
            Box([Label(value="Sample rate"), self.h_da], layout=form_item_layout),
            Box([Label(value="Clip duration (ms)"), self.i_da], layout=form_item_layout),
            Box([Label(value="Window size (ms)"), self.j_da], layout=form_item_layout),
            Box([Label(value="Window stride (ms)"), self.k_da], layout=form_item_layout),
        ]

        self.form_box_data_para = Box(
            form_data_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightblue",
                align_items="stretch",
                width="50%",
            ),
        )

        # button widgets
        self.a_bu = widgets.Button(description="Save Train Setting", layout=Layout(width="30%", height="30px"), button_style="success")
        self.b_bu = widgets.Button(description="Start to Run", layout=Layout(width="30%", height="30px"), button_style="success")

        form_button_items = [self.a_bu, self.b_bu]

        self.form_button = Box(
            form_button_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                # border='solid 3px lightblue',
                align_items="stretch",
                width="50%",
            ),
        )

    def folder_num_check(self, train_loc, dataset_list_check):
        """
        Checks the number of files in each folder within a given directory.
        Args:
            train_loc (str): The path to the main directory containing subfolders.
            dataset_list_check (list): A list of subfolder names to check within the main directory.
        Returns:
            int: Returns 1 if any subfolder contains fewer than 15 files, otherwise returns 0.
        """
        for fld_name in dataset_list_check:
            check_fld = os.path.join(train_loc, fld_name)
            length = len([entry for entry in os.listdir(check_fld) if os.path.isfile(os.path.join(check_fld, entry))])
            if 15 > length:  # need < 15
                return 1
        return 0

    def create_command(self, cm_list):
        """
        Creates a command dictionary from a list of command-line arguments and writes it to a file.
        Args:
            cm_list (list): A list of command-line argument values.
        Returns:
            int: Always returns 0.
        """
        argument_list = [
            "--model_architecture",
            "--testing_percentage",
            "--validation_percentage",
            "--how_many_training_steps",
            "--learning_rate",
            "--eval_step_interval",
            "-batch_size",
            "--model_size_info",
            "--wanted_words",
            "--summaries_dir",
            "--train_dir",
            "--dct_coefficient_count",
            "--data_exist",
            "--background_volume",
            "--background_frequency",
            "--silence_percentage",
            "--unknown_percentage",
            "--time_shift_ms",
            "--sample_rate",
            "--clip_duration_ms",
            "--window_size_ms",
            "--window_stride_ms",
        ]
        cm_dict = OrderedDict()

        for idx, val in enumerate(cm_list):
            if argument_list[idx] == "--model_size_info":  # transfer from single string to list format
                cm_dict[argument_list[idx]] = val.split(",")
            else:
                cm_dict[argument_list[idx]] = val
        print(cm_dict)

        with open("train_cmd.txt", "w", encoding="utf-8") as f:  # save the complete command for train.py
            for key, value in cm_dict.items():

                if isinstance(value, list): 
                    f.write(f"{key} ")
                    for _, val in enumerate(value):
                        f.write(f"{val} ")
                else:
                    f.write(f"{key} {value} ")

        return 0

    def show_main(self):
        """
        Displays the main interface for setting training parameters and running the training process.
        This method creates an interactive UI using Jupyter widgets to allow users to configure training
        parameters or use default settings. It includes an accordion with sections for "Train Setting" 
        and "Data Setting". Users can save the training settings or start the training process.
        """

        intro_text = "Please Choose the parameters of the training or using the default"
        html_widget = widgets.HTML(value=f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(html_widget)

        # Create an accordion and put the 2 boxes
        accordion = widgets.Accordion(children=[self.form_box_train_para, self.form_box_data_para]).add_class("parentstyle")
        # Add a custom style tag to the notebook, you can use dev tool to inspect the class names
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, "Train Setting")
        accordion.set_title(1, "Data Setting")

        output_widgets = widgets.Output(layout=Layout(border="1px solid green"))

        def act_para(*, model, test_per, vali_per, steps, lr, step_inter, batch, dims, outputs, sum_dir, train_dir, dct_coe, data_b_da, b_vol, b_freq, silence, unk, t_sft, rate, dura, win_size, win_str):
            #toggle_train_save = widgets.ToggleButton(description="Save Train Setting", layout=Layout(width="30%", height="30px"), button_style="success")
            #toggle_run = widgets.ToggleButton(description="Start to Run", layout=Layout(width="30%", height="30px"), button_style="success")

            #out = widgets.Output(layout=Layout(border="1px solid green"))

            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()

            #def para_process(obj):
            #    with out:
            #        clear_output()
            #        if obj["new"]:
            #            self.create_command(
            #                [
            #                    model,
            #                    test_per,
            #                    vali_per,
            #                    steps,
            #                    lr,
            #                    step_inter,
            #                    batch,
            #                    dims,
            #                    outputs,
            #                    sum_dir,
            #                    train_dir,
            #                    dct_coe,
            #                    data_b_da,
            #                    b_vol,
            #                    b_freq,
            #                    silence,
            #                    unk,
            #                    t_sft,
            #                    rate,
            #                    dura,
            #                    win_size,
            #                    win_str,
            #                ]
            #            )
#
            #            text0 = "The training setting is finish and saved"
            #            html0 = widgets.HTML(value=f"<b><font color='lightblue'><font size=2>{text0}</b>")
            #            display(html0)
#
            #        else:
            #            print("re-start...")

            #def run(obj):
            #    with out:
            #        clear_output()
            #        if obj["new"]:
            #            self.run_test()
            #        else:
            #            print("stop")

            #toggle_train_save.observe(para_process, "value")
            #toggle_run.observe(run, "value")
            #display(toggle_train_save, toggle_run)
            #display(out)


        out_inter = widgets.interactive_output(
            act_para,
            {
                "model": self.a_ta,
                "test_per": self.b_ta,
                "vali_per": self.c_ta,
                "steps": self.d_ta,
                "lr": self.e_ta,
                "step_inter": self.f_ta,
                "batch": self.g_ta,
                "dims": self.h_ta,
                "outputs": self.i_ta,
                "sum_dir": self.j_ta,
                "train_dir": self.k_ta,
                "dct_coe": self.a_da,
                "data_b_da": self.b_da,
                "b_vol": self.c_da,
                "b_freq": self.d_da,
                "silence": self.e_da,
                "unk": self.f_da,
                "t_sft": self.g_da,
                "rate": self.h_da,
                "dura": self.i_da,
                "win_size": self.j_da,
                "win_str": self.k_da,
            },
        )

        display(accordion, self.form_button, out_inter)
        display(output_widgets)

        def on_button_clicked_save_train_set(b):
            with output_widgets:
                clear_output()
                self.create_command(
                    [
                        self.a_ta.value,
                        self.b_ta.value,
                        self.c_ta.value,
                        self.d_ta.value,
                        self.e_ta.value,
                        self.f_ta.value,
                        self.g_ta.value,
                        self.h_ta.value,
                        self.i_ta.value,
                        self.j_ta.value,
                        self.k_ta.value,
                        self.a_da.value,
                        self.b_da.value,
                        self.c_da.value,
                        self.d_da.value,
                        self.e_da.value,
                        self.f_da.value,
                        self.g_da.value,
                        self.h_da.value,
                        self.i_da.value,
                        self.j_da.value,
                        self.k_da.value,
                    ]
                )
                text0 = "The training setting is finish and saved"
                html0 = widgets.HTML(value=f"<b><font color='lightblue'><font size=2>{text0}</b>")
                display(html0)
        self.a_bu.on_click(on_button_clicked_save_train_set)

        def on_button_clicked_train(b):
            with output_widgets:
                clear_output()
                self.run_test()
    
        self.b_bu.on_click(on_button_clicked_train)    

    def run_test(self):  # run the mainprogram
        """
        Executes the main program for training.
        This method reads a command from a file, processes it, and runs the training
        program based on the provided command. It also checks the dataset's validity
        before proceeding with the training.
        """
        save_cmd_filename = "train_cmd.txt"
        with open(save_cmd_filename, "r", encoding="utf-8") as f:  # save the complete command for train.py
            train_cmd_line = f.read()
        cmd_list = train_cmd_line.split()
        print(cmd_list)

        dataset_list_check = []

        for idx, val in enumerate(cmd_list):
            if val == "False":
                print("change to bool")
                cmd_list[idx] = False
            if val == "--wanted_words":  # get the dataset's name
                dataset_list_check = cmd_list[idx + 1].split(",")

        if cmd_list != []:
            print("read the train commands!")
        else:
            print("The train_cmd.txt is empty!")

        flags, _ = parser.parse_known_args(args=cmd_list)

        dataset_loc_for_check = os.path.join(os.getcwd(), "tmp", "speech_dataset")

        if not self.b_da.value:
            train(flags, save_cmd_filename)
            print("Finish")
        elif self.folder_num_check(dataset_loc_for_check, dataset_list_check):  # if any files < 15, don't run training
            print("The data is not enough, please > 15 files in each label folder.")
            print(f"The dataset path: {dataset_loc_for_check}")
        else:
            train(flags, save_cmd_filename)
            print("Finish")

# Run Section
---
- The detail description of all the parameters is here [meaning](#id-PD)
- Please download the google train data at first time ==> click`Data Setting` tab, and unclick the `Data exist`

In [5]:
act = InitTrainWidgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the parameters of the training or using the…

Accordion(children=(Box(children=(Box(children=(Label(value='Model Architecture'), Dropdown(options=('ds_cnn',…

Box(children=(Button(button_style='success', description='Save Train Setting', layout=Layout(height='30px', wi…

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…

<a id="id-PD"></a>
# Parameter Description
---
- This notebook is basing on https://github.com/ARM-software/ML-examples/tree/main/tflu-kws-cortex-m.

## Train Setting
- `Model Architecture`: What model architecture to use.
- `Testing percentage`: What percentage of wavs as a test set.
- `Validation percentage`: What percentage of wavs as a validation set.
- `Training steps`: How many training loops to run. It matches with the learning rates.
- `Learning rates`: How large a learning rate to use when training. It matches with the training steps.
- `Eval step interval`: How often to evaluate the training results.
- `Batch size`: How many items to train with at once.
- `Model size (dimension)`: Model dimensions - different for various models. For more detail, please check the `train_commands.txt`.
- `Wanted words`: Words to use (others will be added to an unknown label).
- `Summaries directory`: Where to save summary logs for TensorBoard.
- `Train directory`: Directory to write event logs and checkpoint(The trained model and weights).

## Data Setting
- `DCT coefficient count`: How many bins to use for the MFCC fingerprint
- `Data exist`: True will skip download and tar the default tensorflow's speech dataset. (Notice)When you first play this notebook, please unclick it for downlowing the train dataset at first time.
- `Background volume`: How loud the background noise should be, between 0 and 1.
- `Background frequency`: How many of the training samples have background noise mixed in.
- `Silence percentage`: How much of the training data should be silence.
- `Unknown percentage`: How much of the training data should be unknown words.
- `Time shift (ms)`: Range to randomly shift the training audio by in time.
- `Sample rate`: Expected sample rate of the wavs.
- `Clip duration (ms)`: Expected duration in milliseconds of the wavs.
- `Window size (ms)`: How long each spectrogram timeslice is.
- `Window stride (ms)`: Window stride in samples for calculating spectrogram.